In [1]:
from qiskit_algorithms.utils import algorithm_globals

In [2]:
import os
from PIL import Image
import numpy as np

# Path to the train and test folders
train_path = './music/'
test_path = './music_test/'

# Load train images and labels
train_images = []
train_labels = []
for class_folder in os.listdir(train_path):
    class_path = os.path.join(train_path, class_folder)
    if os.path.isdir(class_path):
        label = int(class_folder)  # Assuming folder names are integers representing class labels
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            image = Image.open(image_path)
            image_array = np.asarray(image)
            train_images.append(image_array)
            train_labels.append(label)

# Load test images and labels
test_images = []
test_labels = []
for class_folder in os.listdir(test_path):
    class_path = os.path.join(test_path, class_folder)
    if os.path.isdir(class_path):
        label = int(class_folder)  # Assuming folder names are integers representing class labels
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            image = Image.open(image_path)
            image_array = np.asarray(image)
            test_images.append(image_array)
            test_labels.append(label)
            
            
# Convert lists to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [14]:
train_images.shape

(160, 288, 432, 4)

In [15]:
train_labels.shape

(160,)

In [3]:
x_train = np.float32(train_images.reshape(-1, train_images.shape[1] * train_images.shape[2])) / 255.
y_train = train_labels

x_test = np.float32(test_images.reshape(-1, test_images.shape[1] * test_images.shape[2])) / 255.
y_test = test_labels

In [4]:
x_train.shape

(640, 124416)

In [13]:
y_train.shape

(160,)

In [5]:
import numpy as np
from sklearn.decomposition import PCA

def min_max_scaling(x, new_min, new_max):
    x_min, x_max = x.min(), x.max()
    return (x - x_min) / (x_max - x_min) * (new_max - new_min) + new_min

# Assuming train_images and test_images are loaded correctly
# Adjust the shape of the images accordingly

# Assuming train_labels and test_labels are loaded correctly
# Adjust accordingly if your labels are named differently

# Check the shape of train_images to understand its structure

# Perform PCA
n_components = 2
pca = PCA(n_components)

x_train = pca.fit_transform(x_train)
cumsum = np.cumsum(pca.explained_variance_ratio_ * 100)[-1]
print("Cumulative sum on train :", cumsum)

x_test = pca.transform(x_test)
cumsum = np.cumsum(pca.explained_variance_ratio_ * 100)[-1]
print("Cumulative sum on test :", cumsum)

# # Perform min-max scaling
# x_train = min_max_scaling(x_train, 0, 2 * np.pi)
# x_test = min_max_scaling(x_test, 0, 2 * np.pi)


Cumulative sum on train : 91.21138
Cumulative sum on test : 91.21138


In [6]:
x_train.shape

(640, 2)

In [7]:
def parity(x):
    return "{:b}".format(x).count("1") % 2

In [8]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap
from qiskit.circuit.library import RealAmplitudes

num_inputs = n_components
feature_map = ZZFeatureMap(num_inputs)
ansatz = RealAmplitudes(num_inputs, reps=1)

circuit = QuantumCircuit(num_inputs)
circuit.compose(feature_map, inplace=True)
circuit.compose(ansatz, inplace=True)

In [9]:
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit.primitives import Sampler

sampler = Sampler()


sampler_qnn = SamplerQNN(
    circuit=circuit,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    interpret=parity,
    output_shape=2,
    sampler=sampler,
)

In [10]:
initial_point = algorithm_globals.random.random(ansatz.num_parameters)

In [11]:
from qiskit_algorithms.optimizers import COBYLA

In [12]:
from qiskit_machine_learning.algorithms import NeuralNetworkClassifier

classifier = NeuralNetworkClassifier(
    neural_network=sampler_qnn,
    loss="cross_entropy",
    optimizer=COBYLA(maxiter=100),
    initial_point=initial_point,
)
classifier.fit(x_train, y_train)

QiskitMachineLearningError: "Shapes don't match, predict: (640,), target: (160,)!"

In [ ]:
classifier.score(x_train, y_train)

In [ ]:
classifier.score(x_test, y_test)